In [1]:
from plot import *
from fft import *
from maths import rmse, mae, linear_interpolate, cosine_interpolate, catmull_rom_interpolate, cubic_interpolate, hermite_interpolate


In [2]:
def fit_interpolated_dft2_at_time(filename: str,
                                  variable: str,
                                  time: int,
                                  skip_levels: int = 2,
                                  interpolation: callable = linear_interpolate,
                                  **kwargs):
    data = load_variable_at_time(filename, variable, time)
    prediction = np.zeros((36, 361, 576), dtype="float32")

    preds = []
    levels = sorted(set(range(0, 36, skip_levels)).union({35}))
    for lev in tqdm(levels):
        fft = dft2_at_time_and_level(data[lev], **kwargs)
        preds.append(idft2_at_time_and_level(*fft))

    for lev in range(36):
        if lev in levels:
            prediction[lev] = preds[levels.index(lev)]
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])
        prediction[lev] = interpolation(preds, pred_idx, t)

    data = data.astype("float32")
    print(f"Original Stdev: {data.std()} m/s")
    print(f"Predicted RMSE: {rmse(data, prediction)} m/s")
    print(f"Predicted MAE:  {mae(data, prediction)} m/s")
    print(f"Size/time: {sum(el.nbytes for el in fft) * len(preds) / (1000 ** 2)} MB")
    print(f"Size/day: {sum(el.nbytes for el in fft) * len(preds) * 8 / (1000 ** 2)} MB")
    print(f"Size/year: {sum(el.nbytes for el in fft) * len(preds) * 8 * 365 / (1000 ** 2)} MB")


In [23]:
fit_interpolated_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                              time=0, quantile=0.99044, skip_levels=4, interpolation=linear_interpolate)


  0%|          | 0/10 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.9375601749181541 m/s
Predicted MAE:  0.5611450672149658 m/s
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB


In [24]:
fit_interpolated_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                              time=0, quantile=0.99044, skip_levels=4, interpolation=cosine_interpolate)


  0%|          | 0/10 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.9865887288185732 m/s
Predicted MAE:  0.59920334815979 m/s
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB


In [3]:
fit_interpolated_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                              time=0, quantile=0.99044, skip_levels=4, interpolation=cubic_interpolate)


  0%|          | 0/10 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.810514187440948 m/s
Predicted MAE:  0.5288682579994202 m/s
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB


In [4]:
fit_interpolated_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                              time=0, quantile=0.99044, skip_levels=4, interpolation=catmull_rom_interpolate)


  0%|          | 0/10 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.8263357734885226 m/s
Predicted MAE:  0.5195013880729675 m/s
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB


In [3]:
fit_interpolated_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                              time=0, quantile=0.99044, skip_levels=4, interpolation=hermite_interpolate)


  0%|          | 0/10 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.8263357734885226 m/s
Predicted MAE:  0.5195013880729675 m/s
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB
